In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np

class NeuralNet2(nn.Module):
    def __init__(self, input_size, output_size):
        super(NeuralNet2, self).__init__()
        self.fc1 = nn.Linear(input_size, 1024)
        self.fc2 = nn.Linear(1024, 512)
        self.fc3 = nn.Linear(512, output_size)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x

class SiameseNetwork(nn.Module):
    def __init__(self, input_size, output_size):
        super(SiameseNetwork, self).__init__()
        self.shared_net = NeuralNet2(input_size, output_size)

    def forward_once(self, x):
        return self.shared_net(x)

    def forward(self, input1, input2):
        output1 = self.forward_once(input1)
        output2 = self.forward_once(input2)
        return output1, output2

class ContrastiveLoss(nn.Module):
    def __init__(self, margin=1.0):
        super(ContrastiveLoss, self).__init__()
        self.margin = margin

    def forward(self, output1, output2, label):
        euclidean_distance = F.pairwise_distance(output1, output2)
        loss_contrastive = torch.mean((1-label) * torch.pow(euclidean_distance, 2) +
                                      (label) * torch.pow(torch.clamp(self.margin - euclidean_distance, min=0.0), 2))
        return loss_contrastive


In [2]:
def generate_pairs(data, labels, num_pairs):
    pairs = []
    pair_labels = []
    num_samples = len(data)

    for _ in range(num_pairs):
        # Generate similar pair
        idx1, idx2 = np.random.choice(num_samples, 2, replace=False)
        if np.array_equal(labels[idx1], labels[idx2]):
            pairs.append((data[idx1], data[idx2]))
            pair_labels.append(1)
        else:
            pairs.append((data[idx1], data[idx2]))
            pair_labels.append(0)

    return pairs, pair_labels

# Example dataset
num_samples = 100
input_size = 3000
output_size = 7

data = np.random.rand(num_samples, input_size)  # 100 samples of 3000-dim features
labels = np.random.randint(0, 2, (num_samples, output_size))  # 100 labels of 7-dim arrays

num_pairs = 200
pairs, pair_labels = generate_pairs(data, labels, num_pairs)

input1 = torch.tensor([pair[0] for pair in pairs], dtype=torch.float32)
input2 = torch.tensor([pair[1] for pair in pairs], dtype=torch.float32)
pair_labels = torch.tensor(pair_labels, dtype=torch.float32)


C:\Users\kevin\AppData\Local\Temp\ipykernel_2596\1202294810.py:29: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\torch\csrc\utils\tensor_new.cpp:277.)
  input1 = torch.tensor([pair[0] for pair in pairs], dtype=torch.float32)


In [3]:
siamese_net = SiameseNetwork(input_size, output_size)
criterion = ContrastiveLoss()
optimizer = torch.optim.Adam(siamese_net.parameters(), lr=0.001)

num_epochs = 10

for epoch in range(num_epochs):
    output1, output2 = siamese_net(input1, input2)
    loss = criterion(output1, output2, pair_labels)
    
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    print(f'Epoch {epoch+1}/{num_epochs}, Loss: {loss.item()}')


Epoch 1/10, Loss: 0.02594587951898575
Epoch 2/10, Loss: 0.029370969161391258
Epoch 3/10, Loss: 0.01835859939455986
Epoch 4/10, Loss: 0.013922043144702911
Epoch 5/10, Loss: 0.012045676819980145
Epoch 6/10, Loss: 0.011446717195212841
Epoch 7/10, Loss: 0.011026685126125813
Epoch 8/10, Loss: 0.010243542492389679
Epoch 9/10, Loss: 0.009952994994819164
Epoch 10/10, Loss: 0.009806645102798939


In [4]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_curve, auc

# Forward pass for evaluation
output1, output2 = siamese_net(input1, input2)

# Calculate Euclidean distances
distances = F.pairwise_distance(output1, output2)

# Determine an optimal threshold (for simplicity, let's use 0.5 here)
threshold = 0.5

# Predict labels based on the threshold
predicted_labels = (distances < threshold).float()

# Convert tensors to numpy arrays for metric calculation
pair_labels_np = pair_labels.cpu().numpy()
predicted_labels_np = predicted_labels.cpu().numpy()

# Calculate evaluation metrics
accuracy = accuracy_score(pair_labels_np, predicted_labels_np)
precision = precision_score(pair_labels_np, predicted_labels_np)
recall = recall_score(pair_labels_np, predicted_labels_np)
f1 = f1_score(pair_labels_np, predicted_labels_np)

print(f'Accuracy: {accuracy:.4f}')
print(f'Precision: {precision:.4f}')
print(f'Recall: {recall:.4f}')
print(f'F1-Score: {f1:.4f}')


Accuracy: 0.0150
Precision: 0.0102
Recall: 0.5000
F1-Score: 0.0199
